In [1]:
import pickle
from copy import deepcopy
import numpy as np

class data_object(object):
    def __init__(self, name):
        self.name = name

In [2]:
master_collector = {}

for i in range(1,1001):
    rand_input = data_object('{}'.format(i))
    '''
    model input, randomized
    '''
    # The following parameters requires manual input
    rand_input.tray_number = 20
    rand_input.non_reactive_flag = [1,2,3,4,5,6,7]

    # The following parameters is generated manually
    # reset random seeds
    np.random.seed()

    # distillate ratio = 0.05 - 0.15 total condenser liquid
    rand = np.random.rand()
    rand_input.rr_ratio = 0.05 + 0.1*(rand)

    # generate three product flag, randint produces [low,high)
    rand_input.side_draw_flag = {}
    # intermediate 2 - 4
    rand_int = np.random.randint(low=1,high=3)
    rand = np.random.rand()
    rand_input.side_draw_flag.update({rand_int:0.01+0.02*rand})
    # gasoline 5 - 9
    rand_int = np.random.randint(low=3,high=10)
    rand = np.random.rand()
    rand_input.side_draw_flag.update({rand_int:0.1+0.2*rand})
    # diesel 11 - 18
    rand_int = np.random.randint(low=10,high=19)
    rand = np.random.rand()
    rand_input.side_draw_flag.update({rand_int:0.2+0.3*rand})

    # generating temperature profile
    profile_tmp = sorted(np.random.rand(rand_input.tray_number - len(rand_input.non_reactive_flag)))
    rand_input.temperature_flag = {}; n = 0
    for j in range(1,rand_input.tray_number+1):
        if j not in rand_input.non_reactive_flag:
            rand_input.temperature_flag[j] = 220+60*profile_tmp[n]
            n += 1
    
    # generating feed profile, total = 10 kmol/s
    profile_tmp = {j:np.random.rand() for j in range(1,rand_input.tray_number+1) if j not in rand_input.non_reactive_flag}
    total_tmp = sum(profile_tmp[j] for j in profile_tmp)
    rand_input.feed_flag = {j:0.01 + (10-0.01*len(profile_tmp))*profile_tmp[j]/total_tmp for j in range(1,rand_input.tray_number+1) if j not in rand_input.non_reactive_flag}
    rand_input.feed_flag.update({j:0 for j in rand_input.non_reactive_flag})

    # generating catalyst profile, total = 30000 kg
    profile_tmp = {j:np.random.rand() for j in range(1,rand_input.tray_number+1) if j not in rand_input.non_reactive_flag}
    total_tmp = sum(profile_tmp[j] for j in profile_tmp)
    rand_input.catalyst_flag = {j:100 + (30000-100*len(profile_tmp))*profile_tmp[j]/total_tmp for j in range(1,rand_input.tray_number+1) if j not in rand_input.non_reactive_flag}
    rand_input.catalyst_flag.update({j:0 for j in rand_input.non_reactive_flag})
    
    master_collector[i] = rand_input

In [3]:
with open('random_states_3.pickle','wb') as f:
    pickle.dump(master_collector,f)

In [4]:
master_collector[1].__dict__

{'name': '1',
 'tray_number': 20,
 'non_reactive_flag': [1, 2, 3, 4, 5, 6, 7],
 'rr_ratio': 0.06911659195271289,
 'side_draw_flag': {1: 0.028480832319495213,
  5: 0.22095272848300737,
  13: 0.4154237543986452},
 'temperature_flag': {8: 224.86464707262323,
  9: 228.34170740050627,
  10: 232.97773373004543,
  11: 242.65185729644014,
  12: 244.03628351660547,
  13: 253.3304881850261,
  14: 258.15005783741304,
  15: 264.9494832828836,
  16: 265.16657805354885,
  17: 267.39606803165304,
  18: 272.7302660670859,
  19: 276.3083822774625,
  20: 279.6256244943893},
 'feed_flag': {8: 0.40580137981021475,
  9: 1.2173863527396152,
  10: 0.6646418781725827,
  11: 1.1942533187028352,
  12: 0.9732100385902774,
  13: 0.373782438003868,
  14: 1.3576179859632778,
  15: 0.5549213659057607,
  16: 0.4674659692015652,
  17: 1.1042191168261464,
  18: 0.773793268387047,
  19: 0.30972496805001964,
  20: 0.6031819196467911,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0},
 'catalyst_flag': {8: 1778.1304

# Construct list of commands for bash parallel

In [1]:
with open('command_line.lst','w') as f:
    for i in range(1,1001):
        f.write('./multi_start_onestep.py random_states_3.pickle {0} master_log_{0}.txt\n'.format(i))

# Construct list of scp commands for copying models over

In [3]:
with open('copy_model.lst','w') as f:
    for i in range(801,1001):
        f.write('scp -i /Users/naienh/Downloads/test1.pem ec2-user@ec2-34-209-25-24.us-west-2.compute.amazonaws.com:/home/ec2-user/GitHub/showcase/3rd_level/log/model/Preset_Case_{:}_pf.pickle /Users/naienh/Desktop/model\n'.format(i))